In [172]:
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor

from tqdm import tqdm

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')



In [173]:
## custom library

import eda_util as eutil
import submission_config as subconfig
import submission_util as subutil

In [174]:
pd.set_option('display.float_format', lambda x: f'{x:,g}')

In [175]:
BASE_PATH = subconfig.BASE_PATH
DATA_PATH = subconfig.DATA_PATH

OUTPUT_PATH = subconfig.OUTPUT_PATH

In [176]:
krx_df = pd.read_csv(subconfig.krx_df_PATH)

In [177]:
krx_df.columns = ['date', 'code', 'name', 'volume', 'open', 'high', 'low', 'close']

In [178]:
krx_df['date'] = pd.to_datetime(krx_df['date'], format='%Y%m%d')

In [179]:
krx_df

,date,code,name,volume,open,high,low,close
0,2021-06-01,A060310,3S,166690,2890,2970,2885,2920
1,2021-06-01,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,2021-06-01,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,2021-06-01,A054620,APS,462544,14600,14950,13800,14950
4,2021-06-01,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,2023-05-30,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,2023-05-30,A000540,흥국화재,50218,3250,3255,3195,3215
987997,2023-05-30,A003280,흥아해운,130664,1344,1395,1340,1370
987998,2023-05-30,A037440,희림,141932,9170,9260,9170,9200


In [180]:
return_df = pd.read_pickle(subconfig.return_df_PATH)
close_df = pd.read_pickle(subconfig.adjclose_df_PATH)
open_df = pd.read_pickle(subconfig.adjopen_df_PATH)
high_df = pd.read_pickle(subconfig.adjhigh_df_PATH)
low_df = pd.read_pickle(subconfig.adjlow_df_PATH)

In [181]:
## date list

holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

In [182]:
TRAIN_START = pd.to_datetime(subconfig.TRAIN_START, format='%Y-%m-%d')
SIMOS_END = pd.to_datetime(subconfig.SIMOS_END, format='%Y-%m-%d')

In [183]:
tradingdays = tradingdays[(tradingdays >= TRAIN_START) & (tradingdays <= SIMOS_END)]

In [184]:
dacon_sid_list = [ii[1:] for ii in krx_df['code'].unique()]

In [185]:
return_df = return_df.loc[tradingdays, :].dropna(axis='columns', how='all')
return_df = return_df.loc[:, dacon_sid_list]

close_df = close_df.loc[tradingdays, :].dropna(axis='columns', how='all')
close_df = close_df.loc[:, dacon_sid_list]

In [186]:
open_df = open_df.loc[tradingdays, :].dropna(axis='columns', how='all')
open_df = open_df.loc[:, dacon_sid_list]

high_df = high_df.loc[tradingdays, :].dropna(axis='columns', how='all')
high_df = high_df.loc[:, dacon_sid_list]

low_df = low_df.loc[tradingdays, :].dropna(axis='columns', how='all')
low_df = low_df.loc[:, dacon_sid_list]

In [187]:
SIMOS_START = subconfig.SIMOS_START
# simOS_END = subconfig.SIMOS_END

In [188]:
volume_df = pd.read_pickle(subconfig.volume_df_PATH)
dollarvolume_df = pd.read_pickle(subconfig.dollarvolume_df_PATH)
marketcap_df = pd.read_pickle(subconfig.marketcap_df_PATH)
market_cat_df = pd.read_pickle(DATA_PATH / 'market_cat_df_20140101_20230705.pickle')

In [189]:
volume_df = volume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
volume_df = volume_df.loc[:, dacon_sid_list]

dollarvolume_df = dollarvolume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
dollarvolume_df = dollarvolume_df.loc[:, dacon_sid_list]

marketcap_df = marketcap_df.loc[tradingdays, :].dropna(axis='columns', how='all')
marketcap_df = marketcap_df.loc[:, dacon_sid_list]

In [190]:
market_cat_inrange = market_cat_df[market_cat_df['trdDd'].isin(tradingdays)]

In [191]:
KOSPI_sid_list = market_cat_inrange[market_cat_inrange['is_KOSPI'] == True]['ISU_SRT_CD'].unique()
KOSDAQ_sid_list = market_cat_inrange[market_cat_inrange['is_KOSDAQ'] == True]['ISU_SRT_CD'].unique()
KONEX_sid_list = market_cat_inrange[market_cat_inrange['is_KONEX'] == True]['ISU_SRT_CD'].unique()

## 20일 이평선에 대한 이격도 구하기

In [192]:
disparity_20_df = close_df.copy()

In [193]:
# Calculate 20-day moving average for each stock
moving_avg_20_df = close_df.rolling(window=20).mean()

In [194]:
for column in disparity_20_df.columns:
    disparity_20_df[column] = (disparity_20_df[column] / moving_avg_20_df[column])

In [195]:
disparity_20_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,1.10938,1.00438,0.977442,0.985385,1.02685,0.907882,0.989511,1.02794,0.868714,1.00654,...,0.962167,0.990196,0.936606,0.990554,0.977709,1.01612,0.979021,1.01869,0.959939,1.03967
2023-06-16,1.0713,1.06219,1.01753,0.973623,1.02113,0.924246,0.978155,1.04295,0.876747,1.01357,...,0.971313,0.99951,0.950306,0.993296,1.0308,1.02886,0.982115,1.0102,0.969743,1.07159
2023-06-19,1.09535,1.04858,1.03018,0.957648,1.00632,0.947083,0.966272,0.997341,0.899117,1.01201,...,0.974619,1.01297,0.945248,0.989505,1.03438,1.01992,0.979226,1.03587,0.997495,1.03954


## 60일 이평선에 대한 이격도 구하기

In [196]:
disparity_60_df = close_df.copy()

In [197]:
# Calculate 60-day moving average for each stock
moving_avg_60_df = close_df.rolling(window=60).mean()

In [198]:
for column in disparity_60_df.columns:
    disparity_60_df[column] = (disparity_60_df[column] / moving_avg_60_df[column])

In [199]:
disparity_60_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,1.25823,0.955904,0.994222,0.876808,1.03384,0.754351,0.961584,1.03707,0.836739,1.02925,...,0.957669,1.00295,0.869193,0.993888,1.01417,1.09522,0.974503,1.037,0.949769,1.12137
2023-06-16,1.22381,1.01419,1.02858,0.862743,1.03029,0.76657,0.950715,1.05426,0.835089,1.0364,...,0.965667,1.01221,0.880609,0.997078,1.06904,1.11074,0.974751,1.02988,0.95975,1.1642
2023-06-19,1.26082,1.00434,1.03582,0.854277,1.01723,0.78349,0.938644,1.00898,0.846715,1.03469,...,0.967317,1.02609,0.876164,0.993272,1.07251,1.10297,0.970359,1.05776,0.987282,1.13427


In [200]:
disparity_3_df = close_df.copy()
# Calculate 60-day moving average for each stock
moving_avg_3_df = close_df.rolling(window=3).mean()

for column in disparity_3_df.columns:
    disparity_3_df[column] = (disparity_3_df[column] / moving_avg_3_df[column])

disparity_3_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-03,0.997706,0.991908,1.01633,0.992045,1.00571,0.979979,1.00236,1.01344,0.992138,0.995397,...,0.991952,0.990019,1.01545,1.02343,1.00418,0.996255,1.00178,1,1.05716,1.00581
2021-06-04,1.01143,0.977765,0.991256,0.987298,0.996575,0.986081,0.998588,0.985637,1.01046,1.00293,...,1.00647,1.02304,1.04921,0.994646,1.01177,0.984551,1.00213,1,0.989063,1.01585
2021-06-07,1.05,0.97307,0.975701,1.02288,0.996,1.0182,1.00188,1.02218,0.992126,1.00334,...,1.00964,1.00594,1.02311,0.981985,0.986815,0.98045,0.99431,1,0.955414,1.00571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,1.0011,1.00265,1.00497,0.996283,1.00538,1.00119,0.998369,1.01115,0.994661,0.997089,...,0.983033,1.00331,0.987034,0.995269,0.988089,1.00777,0.98913,0.976392,0.988433,0.983214
2023-06-16,0.988314,1.04,1.01978,0.988412,1.00307,1.01244,0.990984,1.01501,1.00215,1.00631,...,0.997864,1.0099,1.00359,1,1.03178,1.01653,0.99739,0.989481,1.00338,1.02688
2023-06-19,1.01712,1.01202,1.01957,0.985457,0.991545,1.01155,0.98676,0.977368,1.01068,1.0029,...,1.00375,1.01305,0.997382,0.998813,1.0207,1.00218,0.996855,1.01658,1.02156,1


In [201]:
PORTFOLIO_DATE = subconfig.PORTFOLIO_DATE

RDVADV_WINDOW = subconfig.WINDOWS['rdvadv'] # 20

In [202]:
adv_df = dollarvolume_df.rolling(RDVADV_WINDOW, ).mean().dropna(axis='rows', how='all')

In [203]:
avg_adv_s = adv_df.mean(axis='columns')
avg_rdv_s = dollarvolume_df.iloc[RDVADV_WINDOW:, :].mean(axis='columns')

In [204]:
avg_rdvadv_s = avg_rdv_s / avg_adv_s

In [205]:
ii_rdvadv_df = dollarvolume_df.iloc[RDVADV_WINDOW:, :] / adv_df

In [206]:
normalized_rdvadv_signal_df = ii_rdvadv_df.divide(avg_rdvadv_s, axis='rows')
normalized_rdvadv_signal_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-29,0.415442,0.240279,0.405709,0.726369,0.475821,0.907576,0.4201,1.10324,0.539159,0.755196,...,1.81974,0.798739,0.631614,0.340136,14.3229,0.996211,0.600184,NaN,0.369902,0.630198
2021-06-30,0.314977,0.212304,0.465795,0.931287,0.470661,0.514831,0.648498,1.99712,0.354948,0.501368,...,4.39418,8.5227,0.745765,0.319291,7.08257,0.416926,1.40616,NaN,0.291685,0.618186
2021-07-01,0.399208,0.295054,0.570299,0.366269,0.496024,0.584884,1.26369,3.66506,0.356092,0.701445,...,1.03469,1.22015,0.426687,0.971435,1.91106,0.786424,0.705502,NaN,0.495063,0.955606
2021-07-02,1.02063,0.146543,0.639072,0.410245,0.842014,0.260692,1.97162,1.5627,0.724157,0.519005,...,2.03734,0.881352,0.244086,2.64395,0.377858,0.422299,0.421016,NaN,1.13061,0.666445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,0.292674,0.588688,1.12985,0.124622,0.90052,0.31337,0.973597,1.10088,0.442653,1.00447,...,0.592776,0.71876,0.580354,0.487591,0.570073,0.493214,1.08468,0.789117,0.514062,0.106241
2023-06-16,0.12223,3.09939,1.38265,0.322443,0.710468,0.296359,1.33772,2.43478,0.495513,1.70201,...,0.44873,0.936876,0.771501,0.2652,2.44716,1.00167,0.93063,0.793416,0.350493,0.14148
2023-06-19,0.31029,1.29243,1.28678,0.15679,0.463203,0.425583,1.23098,1.1466,0.351447,0.838827,...,0.849195,1.65108,0.607847,0.452039,2.08872,0.661718,1.23745,0.757414,1.75569,0.0806887


In [213]:
MODEL_TRAIN_START = pd.to_datetime('2021-06-29', format='%Y-%m-%d')

In [214]:
# Your function to calculate SMAPE
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [215]:
results_df = pd.DataFrame(columns=['code'] + ['return_day_' + str(i) for i in range(1, 16)])

preds_df_fin_xgb = pd.DataFrame()
smapes_df_fin_xgb = pd.DataFrame()

preds_df_fin_lgbm = pd.DataFrame()
smapes_df_fin_lgbm = pd.DataFrame()

#preds_df_fin_catboost = pd.DataFrame()
#smapes_df_fin_catboost = pd.DataFrame()

In [216]:
import logging
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

In [ ]:
from prophet import Prophet

smapes_fin_xgb = {}  # Initialize as dictionary
smapes_fin_lgbm = {}  # Initialize as dictionary
#smapes_fin_prophet = {}  # Initialize as dictionary


preds_fin_xgb = {}  # Initialize as dictionary
preds_fin_lgbm = {}  # Initialize as dictionary
#preds_fin_prophet = {}  # Initialize as dictionary

# Iterate over each unique stock
for code in tqdm(dacon_sid_list):
    
    # Filter by stock code
    # Note: All prices are adjusted
    # TODO: Add normalized rdvadv signal to the columns

    train_close = pd.DataFrame(
        data={
            'open': open_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'high': high_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'low': low_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'close': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'close_t-1': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code].shift(1),
            'close_t-3': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code].shift(3),
            'close_t-5': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code].shift(5),
            'dollarvolume': dollarvolume_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'marketcap': marketcap_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'norm_rdvadv': normalized_rdvadv_signal_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'disparity_3' : disparity_3_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            #'disparity_20' : disparity_20_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            #'disparity_60' : disparity_60_df.loc[MODEL_TRAIN_START:SIMOS_START, code]
        }
    )
    train_close = train_close.iloc[5:, :] # nan 있는 1st row 제거 

    # Store original data for reference
    original_data = train_close.copy()

    smapes_xgb = []  # Initialize empty lists
    smapes_lgbm = []  # Initialize empty lists
    #smapes_prophet = []  # Initialize empty lists
    

    preds_xgb = []  # Initialize empty lists
    preds_lgbm = []  # Initialize empty lists
    #preds_prophet = []  # Initialize empty lists
    
    # For each day from 1 to 15
    for day in range(1, 16):
        # Scale data
        X = train_close[:]
        y = train_close['close']
        
        scaler = MinMaxScaler(feature_range=(-1, 1))
        data_scaled = scaler.fit_transform(X)
        data_scaled2 = y
        
        X_train = data_scaled[:-day]
        y_train = data_scaled2[day:]
        X_test = data_scaled[-day]
        
        X_train = X_train[:int(len(X_train) * 0.9)]
        X_val = X_train[int(len(X_train) * 0.9):]
        y_train = y_train[:int(len(y_train) * 0.9)]
        y_val = y_train[int(len(y_train) * 0.9):] 

        # Train XGBoost
        xgb_model = XGBRegressor()
        xgb_model.fit(X_train, y_train)
        vals_xgb = xgb_model.predict(X_val)
        smapes_xgb.append(smape(y_val, vals_xgb))
        
        preds_xgb.append(xgb_model.predict([data_scaled[-day]]))

        # Train LightGBM
        lgbm_model = LGBMRegressor(verbose = -1)
        lgbm_model.fit(X_train, y_train)
        vals_lgbm = lgbm_model.predict(X_val)
        smapes_lgbm.append(smape(y_val, vals_lgbm))
        
        preds_lgbm.append(lgbm_model.predict([data_scaled[-day]]))
        
    
    smapes_fin_xgb[code] = smapes_xgb
    smapes_fin_lgbm[code] = smapes_lgbm
    #smapes_fin_prophet[code] = smapes_prophet

    preds_fin_xgb[code] = preds_xgb
    preds_fin_lgbm[code] = preds_lgbm
    #preds_fin_prophet[code] = preds_prophet

# Convert the dictionaries to DataFrames after the loop
smapes_df_fin_xgb = pd.DataFrame(smapes_fin_xgb)
smapes_df_fin_lgbm = pd.DataFrame(smapes_fin_lgbm)
#smapes_df_fin_prophet = pd.DataFrame(smapes_fin_prophet)

preds_df_fin_xgb = pd.DataFrame(preds_fin_xgb)
preds_df_fin_lgbm = pd.DataFrame(preds_fin_lgbm)
#preds_df_fin_prophet = pd.DataFrame(preds_fin_prophet)


 88%|████████████████████████████████████████████████████████████████████▌         | 1758/2000 [36:05<05:33,  1.38s/it]

In [ ]:
preds_df_fin_lgbm.shape

In [ ]:
final = np.zeros((len(smapes_df_fin_xgb), len(smapes_df_fin_xgb.columns)))

for i in range(0, len(smapes_df_fin_xgb.columns)):
    for j in range(0, len(smapes_df_fin_xgb)):
        weights = [1 / smapes_df_fin_xgb.iloc[j:j+1, i].values[0],
                   
                   1 / smapes_df_fin_lgbm.iloc[j:j+1, i].values[0]]
                  
                   #1 / smapes_df_fin_prophet.iloc[j:j+1, i].values[0]]
        

        weights /= np.sum(weights) 
        

        final[j][i] = weights[0] * preds_df_fin_xgb.iloc[j:j+1, i].values[0] \
                            + weights[1] * preds_df_fin_lgbm.iloc[j:j+1, i].values[0] \
                             #+ weights[2] * preds_df_fin_prophet.iloc[j:j+1, i].values[0]

In [ ]:
final_df = pd.DataFrame(final)
final_values = pd.DataFrame((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0])
final_values_sharpe = -pd.DataFrame(((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0]) / final_df.pct_change().std())

In [ ]:
final_values

In [ ]:
final_values_sharpe.index = dacon_sid_list
final_values_sharpe.columns = ['VALUE']
final_values_sharpe.reset_index(inplace = True)
final_values_sharpe.columns = ['종목코드', 'VALUE']

In [ ]:
final_values_sharpe.set_index('종목코드', inplace = True)

In [ ]:
final_values_sharpe['VALUE']

In [ ]:
alpha_feat_insoo = subutil.Submission(
    alpha_series=final_values_sharpe['VALUE'],
    alpha_name='alpha_feat_insoo_lagged_ReverseSharpe_disparity_added',
)

In [ ]:
alpha_feat_insoo.get_rank(export_path=OUTPUT_PATH)

In [ ]:
import submission_util as subutil
import submission_config as subconfig

submission = subconfig.OUTPUT_PATH / 'alpha_feat_insoo_lagged_ReverseSharpe_no_disparity_prophet.csv' # ...or you can just type the path here

test1 = subutil.Score(submission, 'my_alpha_name')
test1.get_Sharpe()